# Overlap between chemical libraries

In [ ]:
import pandas as pd

from rdkit import Chem
from rdkit.Chem import MolToInchiKey
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumRings

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
chebi = PandasTools.LoadSDF('chebi.sdf')
Antibacterial = PandasTools.LoadSDF('Antibacterial-Library-13880.sdf')
drugRepurposing_hub = pd.read_csv('drug_repurposing_hub.csv')
drugbank_smiles = pd.read_csv('drugbank_smiles.csv')

In [ ]:
chebi

In [ ]:
chembl = pd.read_csv('/home/marina/Documents/DB/reference_cs/chembl_bitbirch_representatives_3.csv')

In [ ]:
Antibacterial.columns

Molecule columns:
* chebi['INHIKEY']
* Antibacterial['InChI Key']
* drugRepurposing_hub['smiles']
*  drugbank_smiles['Smiles']
* chembl['SMILES']

In [ ]:
print("## 🔢 Molecule Counts in Each Dataset")
print(f"| {'Dataset':<30} | {'Number of Molecules':<20} |")
print(f"|{'-'*31}|{'-'*22}|")
print(f"| {'ChEBI':<30} | {len(chebi):<20} |")
print(f"| {'Antibacterial Library':<30} | {len(Antibacterial):<20} |")
print(f"| {'Drug Repurposing Hub':<30} | {len(drugRepurposing_hub):<20} |")
print(f"| {'DrugBank SMILES':<30} | {len(drugbank_smiles):<20} |")
print(f"| {'Filtered ChEMBL':<30} | {len(chembl):<20} |")

## Add Inchikeys to missing libraries

In [ ]:
from rdkit import Chem
from rdkit.Chem.rdinchi import MolToInchiKey
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdinchi import MolToInchiKey
from rdkit.Chem.MolStandardize import rdMolStandardize
import pandas as pd

# --- Initialize RDKit Standardizers ---
# These are heavy objects, so we create them once globally
lfc = rdMolStandardize.LargestFragmentChooser()
normalizer = rdMolStandardize.Normalizer()
uncharger = rdMolStandardize.Uncharger()

def standardize_and_get_inchikey(mol, strip_stereo=True):
    """
    Standardize a molecule:
      - Largest fragment (strips salts/solvents)
      - Normalize & Uncharge
      - Optional stereochemistry stripping
    Returns (mol_clean, inchikey, smiles)
    """
    if mol is None:
        return None, None, None

    try:
        # 1. Largest fragment (remove salts/solvents)
        mol = lfc.choose(mol)

        # 2. Cleanup + normalize + uncharge
        mol = rdMolStandardize.Cleanup(mol)
        mol = normalizer.normalize(mol)
        mol = uncharger.uncharge(mol)

        # 3. Sanitize structure
        Chem.SanitizeMol(mol)

        # 4. Optionally collapse stereochemistry
        if strip_stereo:
            Chem.RemoveStereochemistry(mol)

        # 5. Generate InChIKey and Canonical SMILES
        ik = MolToInchiKey(mol)
        smiles = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=not strip_stereo)
        
        return mol, ik, smiles
    except Exception as e:
        return None, None, None

def process_dataframe_chemistry(df, smiles_col, new_inchikey_col, strip_stereo=True):
    """
    Helper to apply standardization to a specific dataframe.
    """
    print(f"Processing {smiles_col}...")
    
    # Create initial Mol objects
    df['Mol_Temp'] = df[smiles_col].apply(lambda x: Chem.MolFromSmiles(str(x)) if pd.notna(x) else None)
    
    # Apply the standardization function
    # This returns a Series of tuples, which we expand into 3 columns
    results = df['Mol_Temp'].apply(lambda m: standardize_and_get_inchikey(m, strip_stereo=strip_stereo))
    
    # Assign results back to dataframe
    _, df[new_inchikey_col], df['Canonical_SMILES'] = zip(*results)
    
    # Cleanup
    df.drop('Mol_Temp', axis=1, inplace=True)
    
    valid_count = df[new_inchikey_col].count()
    print(f"Standardized {valid_count}/{len(df)} molecules.")
    return df

# --- Process drugRepurposing_hub ---
drugRepurposing_hub = process_dataframe_chemistry(
    drugRepurposing_hub,
    smiles_col='smiles',
    new_inchikey_col='INCHIKEY-Canonical'
)

# --- Process drugbank_smiles ---
drugbank_smiles = process_dataframe_chemistry(
    drugbank_smiles,
    smiles_col='Smiles',
    new_inchikey_col='INCHIKEY-Canonical'
)

chebi = process_dataframe_chemistry(
    chebi,
    smiles_col='SMILES',
    new_inchikey_col='INCHIKEY-Canonical'
)
Antibacterial= process_dataframe_chemistry(
    Antibacterial,
    smiles_col='Smile',
    new_inchikey_col='INCHIKEY-Canonical'
)
# chembl= process_dataframe_chemistry(
#     chembl,
#     smiles_col='SMILES',
#     new_inchikey_col='INCHIKEY-Canonical'
# )   


In [ ]:
# --- Standardize Column Names ---

# 2. Antibacterial Library: Rename 'InChI Key' to 'INCHIKEY'
# Antibacterial.rename(columns={'InChI Key': 'INCHIKEY'}, inplace=True)
# print(f"Antibacterial column names after rename: {list(Antibacterial.columns)}")

# --- Create a list of the DataFrames to iterate over for overlap analysis ---
datasets = {
    'ChEBI': chebi,
    'chemdiv': Antibacterial,
    'Drug Repurposing Hub': drugRepurposing_hub,
    'DrugBank SMILES': drugbank_smiles
}

In [ ]:
import itertools

# --- SECTION 1: Overlap with ChEMBL (Your original logic) ---
# chembl_inchikeys = set(chembl['INCHIKEY'].dropna().str.strip())
# N_chembl = len(chembl_inchikeys)

# print(f"\nTotal unique InChIKeys in Filtered ChEMBL: {N_chembl}")
# print("\n## 🔬 Overlap Analysis: Libraries vs. Filtered ChEMBL")
# print(f"| {'Library':<25} | {'Shared Molecules':<18} | {'Jaccard Index (%)':<20} |")
# print(f"|{'-'*26}|{'-'*19}|{'-'*22}|")

# # Store library sets in a dict for the next step
library_sets = {}

for name, df in datasets.items():
    lib_keys = set(df['INCHIKEY-Canonical'].dropna().str.strip())
    library_sets[name] = lib_keys
    
#     shared = lib_keys.intersection(chembl_inchikeys)
#     union = lib_keys.union(chembl_inchikeys)
#     jaccard = (len(shared) / len(union)) * 100 if union else 0
    
#     print(f"| {name:<25} | {len(shared):<18} | {jaccard:.4f}% |")

# --- SECTION 2: Pairwise Library Overlap (New) ---
print("\n## 🤝 Pairwise Library Overlap Analysis")
print(f"| {'Library A':<20} | {'Library B':<20} | {'Shared':<10} | {'Jaccard Index (%)':<20} |")
print(f"|{'-'*22}|{'-'*22}|{'-'*12}|{'-'*22}|")

# Get all unique pairs of libraries
lib_names = list(library_sets.keys())
for name_a, name_b in itertools.combinations(lib_names, 2):
    set_a = library_sets[name_a]
    set_b = library_sets[name_b]
    
    shared = set_a.intersection(set_b)
    union = set_a.union(set_b)
    jaccard = (len(shared) / len(union)) * 100 if union else 0
    
    print(f"| {name_a:<20} | {name_b:<20} | {len(shared):<10} | {jaccard:.4f}% |")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors

# 1. Initialize the Shared Matrix
# (Assuming library_sets is defined in your environment)
names = list(library_sets.keys())
n = len(names)
shared_matrix = pd.DataFrame(np.zeros((n, n), dtype=int), index=names, columns=names)

for i, name_a in enumerate(names):
    for j, name_b in enumerate(names):
        count = len(library_sets[name_a].intersection(library_sets[name_b]))
        shared_matrix.iloc[i, j] = count

# 2. DEFINE YOUR THRESHOLDS
thresholds = [0, 1, 10, 100, 1000, 5000, 10000]

# 3. Choose colors for each bin
colors = ["#fffff0", "#FAD782", "#FAA08B", "#8DC7FA", "#AA96FA", "#50285A", "#50285A"]
cmap = mcolors.ListedColormap(colors)
norm = mcolors.BoundaryNorm(thresholds, cmap.N, extend='max')

# 4. Prepare Masks for Lower Triangle
# np.triu returns the upper triangle of an array.
# k=1 means we start masking 1 unit above the main diagonal.
upper_tri_mask = np.triu(np.ones_like(shared_matrix, dtype=bool), k=1)
diag_mask = np.eye(n, dtype=bool)

# Combined mask for the colored overlap (Mask Upper + Mask Diagonal)
off_diag_mask_to_hide = upper_tri_mask | diag_mask

# Combined mask for the grey diagonal (Mask Upper + Mask Lower)
# This leaves only the diagonal visible.
diag_only_mask_to_hide = upper_tri_mask | ~diag_mask

# 5. Create the Visualization
plt.figure(figsize=(12, 10))

# First Layer: Plot the Shared Counts (Lower Triangle Only)
ax = sns.heatmap(shared_matrix, 
                 mask=off_diag_mask_to_hide, 
                 annot=True, 
                 fmt="d", 
                 cmap=cmap, 
                 norm=norm, 
                 annot_kws={"size": 14},
                 cbar_kws={'label': 'Shared Molecules (Overlap Scale)'},
                 linewidths=1,
                 linecolor='white')

# Second Layer: Plot the Diagonal (Grey)
# We use the same 'ax' to overlay the diagonal on the same plot
sns.heatmap(shared_matrix, 
            mask=diag_only_mask_to_hide, 
            annot=True, 
            fmt="d", 
            annot_kws={"size": 14},
            cmap=mcolors.ListedColormap(['#d3d3d3']), # Light Grey for diagonal
            cbar=False, 
            ax=ax)

# Final Formatting
plt.title("Chemical Overlap", fontsize=16, pad=20)
plt.xlabel("Library", fontsize=12)
plt.ylabel("Library", fontsize=12)

# Optional: Adjust layout to ensure no clipping
plt.tight_layout()
plt.show()

In [ ]:
corr =  df.corr()

# Create a mask
mask = np.triu(np.ones_like(corr, dtype=bool))



In [ ]:

# --- 1. SETUP: Prepare DataFrames and Descriptors ---

# Dictionary of datasets and their SMILES/Molecule column names
# Assuming 'ROMol' is the default column name from PandasTools.LoadSDF for SDF files
datasets_info = {
    'Filtered ChEMBL': {'df': chembl, 'smiles_col': 'SMILES', 'mol_col': 'Mol'},
    'ChEBI': {'df': chebi, 'smiles_col': None, 'mol_col': 'ROMol'},
    'Antibacterial': {'df': Antibacterial, 'smiles_col': None, 'mol_col': 'ROMol'},
    'Drug Repurposing Hub': {'df': drugRepurposing_hub, 'smiles_col': 'smiles', 'mol_col': 'Mol'},
    'DrugBank SMILES': {'df': drugbank_smiles, 'smiles_col': 'Smiles', 'mol_col': 'Mol'}
}

# --- Helper Functions ---

def ensure_mol_column(df, smiles_col, mol_col):
    """Ensure a molecule object column exists for SMILES-based DataFrames."""
    if smiles_col and mol_col not in df.columns:
        print(f"Generating Mol objects for: {smiles_col}")
        df[mol_col] = df[smiles_col].apply(lambda x: Chem.MolFromSmiles(str(x)) if pd.notna(x) else None)
    return df

def calculate_descriptors(df, mol_col):
    """Calculates all required RDKit descriptors."""
    print(f"Calculating descriptors based on column: {mol_col}")
    df['MW'] = df[mol_col].apply(lambda mol: Descriptors.MolWt(mol) if mol else None)
    df['HeavyAtoms'] = df[mol_col].apply(lambda mol: Descriptors.HeavyAtomCount(mol) if mol else None)
    df['NumRings'] = df[mol_col].apply(lambda mol: CalcNumRings(mol) if mol else None)
    df['LogP'] = df[mol_col].apply(lambda mol: Descriptors.MolLogP(mol) if mol else None)
    df['HBD'] = df[mol_col].apply(lambda mol: Descriptors.NumHDonors(mol) if mol else None)
    df['HBA'] = df[mol_col].apply(lambda mol: Descriptors.NumHAcceptors(mol) if mol else None)
    df['RotatableBonds'] = df[mol_col].apply(lambda mol: Descriptors.NumRotatableBonds(mol) if mol else None)
    return df

# --- 2. EXECUTION: Process DataFrames ---

all_data_list = []

for name, info in datasets_info.items():
    df = info['df']
    smiles_col = info['smiles_col']
    mol_col = info['mol_col']

    # 2.1 Ensure Mol column is present for SMILES-based files
    df = ensure_mol_column(df, smiles_col, mol_col)

    # 2.2 Calculate descriptors
    df = calculate_descriptors(df, mol_col)

    # 2.3 Create a subset and add a 'Library' identifier column
    subset_cols = ['MW', 'HeavyAtoms', 'NumRings', 'LogP', 'HBD', 'HBA', 'RotatableBonds']
    temp_df = df[subset_cols].copy()
    temp_df['Library'] = name
    all_data_list.append(temp_df)

# Combine all data into one DataFrame for easy plotting
plot_data = pd.concat(all_data_list, ignore_index=True)



In [ ]:
properties = {
    'MW': 'Molecular Weight',
    'LogP': 'LogP',
    'HeavyAtoms': 'Heavy Atoms',
    'NumRings': 'Number of Rings',
    'HBD': 'HB Donors',
    'HBA': 'HB Acceptors',
    'RotatableBonds': 'Rotatable Bonds'
}


In [ ]:
plot_data

In [ ]:
import pandas as pd

# Assuming 'plot_data' DataFrame is available from the previous run

print("## 🧼 Final Data Cleaning on Plot Data")

# --- Cleaning the Library Column ---
# Strip any leading/trailing whitespace from the Library column
plot_data['Library'] = plot_data['Library'].astype(str).str.strip()

# Check unique values after cleaning (to confirm the fix)
print("Unique Libraries found after cleaning:")
print(plot_data['Library'].unique())


# --- Re-Plotting (Using the corrected, robust code from last turn) ---

# Define the properties to plot and their display names
properties = {
    'MW': 'Molecular Weight',
    'LogP': 'LogP',
    'HeavyAtoms': 'Heavy Atoms',
    'NumRings': 'Number of Rings',
    'HBD': 'HB Donors',
    'HBA': 'HB Acceptors',
    'RotatableBonds': 'Rotatable Bonds'
}

n_plots = len(properties)
n_cols = 2
n_rows = (n_plots + n_cols - 1) // n_cols 

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows)) 
axes = axes.flatten()

# Get the list of unique libraries for the legend title
unique_libraries = plot_data['Library'].unique()

# Use a color palette distinct enough for 5 libraries
palette = sns.color_palette("Set2", len(unique_libraries))
color_map = dict(zip(unique_libraries, palette))


for i, (col, title) in enumerate(properties.items()):
    ax = axes[i]
    
    # Plot the density using the cleaned data
    sns.kdeplot(
        data=plot_data,
        x=col,
        hue='Library',
        fill=False,
        alpha=0.5,
        linewidth=1.5,
        palette=color_map, 
        ax=ax,
        legend=True 
    )
    
    # Add titles and labels
    ax.set_title(f'Distribution of {title}', fontsize=16)
    ax.set_xlabel(title, fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

# Turn off any extra empty subplots
for i in range(n_plots, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()

# Force display of the plot in the notebook
plt.show() 

# Save the plot
plt.savefig('Library_Property_Overlap_Cleaned.png')
print("\n[Image saved as 'Library_Property_Overlap_Cleaned.png']")
print("All distribution plots should now be visible with 5 densities.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'plot_data' DataFrame is available, cleaned, and contains the descriptor columns

print("## 📊 Step 8: Plotting with Zoomed-In Y-Axis for Small Libraries")

# Define the properties to plot
properties = {
    'MW': 'Molecular Weight',
    'LogP': 'LogP',
    'HeavyAtoms': 'Heavy Atoms',
    'NumRings': 'Number of Rings',
    'HBD': 'HB Donors',
    'HBA': 'HB Acceptors',
    'RotatableBonds': 'Rotatable Bonds'
}

n_plots = len(properties)
n_cols = 2
n_rows = (n_plots + n_cols - 1) // n_cols 

# Get unique libraries and color map
unique_libraries = plot_data['Library'].unique()
palette = sns.color_palette("Set2", len(unique_libraries))
color_map = dict(zip(unique_libraries, palette))

# --- Plot Set 1: Full Range (Reference) ---
# This is mainly to see the ChEMBL peak.

fig1, axes1 = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows)) 
axes1 = axes1.flatten()

for i, (col, title) in enumerate(properties.items()):
    ax = axes1[i]
    sns.kdeplot(
        data=plot_data, x=col, hue='Library', fill=False, linewidth=2.0, 
        palette=color_map, ax=ax, legend=True 
    )
    ax.set_title(f'Full Range: Distribution of {title}', fontsize=16)
    ax.set_xlabel(title, fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

for i in range(n_plots, len(axes1)):
    fig1.delaxes(axes1[i])

plt.tight_layout()
plt.show() 
plt.savefig('Property_Overlap_FullRange.png')
print("\n[Image saved as 'Property_Overlap_FullRange.png']")


# --- Plot Set 2: Zoomed-In Y-Axis (Focus on Small Libraries) ---
# We'll use a maximum Y limit of 0.0002 (or 0.0005) to reveal the small libraries.
# You may need to adjust Y_LIMIT based on your exact data.

Y_LIMIT = 0.0015 

fig2, axes2 = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows)) 
axes2 = axes2.flatten()

for i, (col, title) in enumerate(properties.items()):
    ax = axes2[i]
    sns.kdeplot(
        data=plot_data, x=col, hue='Library', fill=False, linewidth=2.0, 
        palette=color_map, ax=ax, legend=True 
    )
    
    # --- KEY CHANGE: Set the Y-axis limit ---
    ax.set_ylim(0, Y_LIMIT)
    
    ax.set_title(f'Zoomed (Y-Max {Y_LIMIT}): Distribution of {title}', fontsize=16)
    ax.set_xlabel(title, fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.7)

for i in range(n_plots, len(axes2)):
    fig2.delaxes(axes2[i])

plt.tight_layout()
plt.show() 
plt.savefig('Property_Overlap_Zoomed.png')
print("\n[Image saved as 'Property_Overlap_Zoomed.png']")
print("Successfully generated two sets of plots: one full range and one zoomed into the low-density area.")

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MolToSmiles

# --- 1. Setup DataFrames and Column Mapping ---

# Define the source DataFrames and their respective SMILES/Mol column names
all_source_data = [
    {'df': chembl,                 'inchikey_col': 'INCHIKEY', 'smiles_col': 'SMILES', 'mol_col': 'Mol', 'name': 'Filtered ChEMBL'},
    {'df': chebi,                  'inchikey_col': 'INCHIKEY', 'smiles_col': None,     'mol_col': 'ROMol', 'name': 'ChEBI'},
    {'df': Antibacterial,          'inchikey_col': 'INCHIKEY', 'smiles_col': None,     'mol_col': 'ROMol', 'name': 'Antibacterial'},
    {'df': drugRepurposing_hub,    'inchikey_col': 'INCHIKEY', 'smiles_col': 'smiles', 'mol_col': 'Mol', 'name': 'Drug Repurposing Hub'},
    {'df': drugbank_smiles,        'inchikey_col': 'INCHIKEY', 'smiles_col': 'Smiles', 'mol_col': 'Mol', 'name': 'DrugBank SMILES'}
]

# --- 2. Helper Function to get SMILES ---

def get_canonical_smiles(row, smiles_col, mol_col):
    """
    Returns the canonical SMILES string from either a SMILES column or a Mol column.
    Handles potential failure if both are None or invalid.
    """
    mol = None
    if smiles_col and pd.notna(row[smiles_col]):
        # Try converting from SMILES string (CSV sources)
        mol = Chem.MolFromSmiles(str(row[smiles_col]))
    elif mol_col in row and row[mol_col] is not None:
        # Use the existing RDKit Mol object (SDF sources)
        mol = row[mol_col]

    if mol:
        # Convert to canonical SMILES
        try:
            return MolToSmiles(mol, isomericSmiles=True, canonical=True)
        except:
            return None
    return None

# --- 3. Build the Master INCHIKEY-SMILES Lookup Table ---

master_lookup_list = []

print("## 🔍 Building INCHIKEY and Canonical SMILES Lookup Table")

for data in all_source_data:
    df = data['df'].copy()
    inchikey_col = data['inchikey_col']
    smiles_col = data['smiles_col']
    mol_col = data['mol_col']
    
    # Generate canonical SMILES for every row
    df['Canonical_SMILES'] = df.apply(
        lambda row: get_canonical_smiles(row, smiles_col, mol_col), axis=1
    )
    
    # Keep only the essential columns for lookup
    lookup_subset = df[[inchikey_col, 'Canonical_SMILES']].copy()
    lookup_subset.rename(columns={inchikey_col: 'INCHIKEY'}, inplace=True)
    
    # Clean and deduplicate the lookup set for this library
    lookup_subset.dropna(subset=['INCHIKEY', 'Canonical_SMILES'], inplace=True)
    lookup_subset.drop_duplicates(subset=['INCHIKEY'], keep='first', inplace=True)
    
    master_lookup_list.append(lookup_subset)

# Combine all lookups and deduplicate the FINAL list by INCHIKEY
smiles_lookup = pd.concat(master_lookup_list, ignore_index=True)
smiles_lookup.drop_duplicates(subset=['INCHIKEY'], keep='first', inplace=True)

print(f"Total unique (INCHIKEY, SMILES) pairs created: {len(smiles_lookup)}")


# --- 4. Merge the Lookup Table with the Integrated Dataset ---

# master_dataset_inchikeys is the integrated list from Step 6 (INCHIKEY, Original_Library)

master_dataset = pd.merge(
    master_dataset_inchikeys,
    smiles_lookup,
    on='INCHIKEY',
    how='left' # Keep all entries from our integrated list
)

# Final cleanup: drop any rows that somehow lost their SMILES during the merge 
# (Should be few, but good practice)
master_dataset.dropna(subset=['Canonical_SMILES'], inplace=True)

# Rename for clarity
master_dataset.rename(columns={'Canonical_SMILES': 'SMILES'}, inplace=True)

print(f"\n--- Final Master Dataset Creation Complete ---")
print(f"Final Master Dataset Size (with SMILES): {len(master_dataset)}")
print("Final Master Dataset Head:")
print(master_dataset.head())

In [ ]:
import pandas as pd
import os

# Assuming the 'master_dataset' DataFrame is available from the previous step 9

# --- 1. Prepare the DataFrame for Saving ---
# Select only the 'SMILES' column
smiles_only_df = master_dataset[['SMILES']].copy()

# --- 2. Define the File Name ---
output_filename = 'master_dataset_smiles.csv'

# --- 3. Save to CSV ---
smiles_only_df.to_csv(output_filename, index=False)

print(f"\n## ✅ Integration Complete: Master SMILES Saved")
print(f"The final list of unique SMILES has been successfully saved to: {output_filename}")
print(f"File contents verification (first 5 lines):")

# Verification: Print the head of the saved file to confirm format
with open(output_filename, 'r') as f:
    for i in range(5):
        print(f.readline().strip())

print(f"\nTotal SMILES saved: {len(smiles_only_df)}")